## Evaluating a vectorial function on CPU and GPU

In [13]:
import numpy as np
import time

# GPU libs
import cupy as cp
from numba import njit, vectorize, cuda

# Reproducibilidad
np.random.seed(42)

# Tamaño (como en la práctica)
size = 5_000_000

# Datos en CPU (float32 = simple precisión)
a_cpu = np.random.rand(size).astype(np.float32)
b_cpu = np.random.rand(size).astype(np.float32)

# Constantes (float32)
a = np.float32(3.5)
b = np.float32(2.8)
c = np.float32(10.0)

print("CPU dtypes:", a_cpu.dtype, b_cpu.dtype, a.dtype, b.dtype, c.dtype)

# Info GPU
print("CuPy version:", cp.__version__)
print("GPU device:", cp.cuda.runtime.getDeviceProperties(0)["name"].decode())

CPU dtypes: float32 float32 float32 float32 float32
CuPy version: 13.6.0
GPU device: NVIDIA GeForce GTX 1080


In [15]:
import numpy as np
import cupy as cp

def gpu_time_ms(fn, n_repeat=10, n_warmup=3):
    """Mide tiempo en GPU con CUDA Events (ms)."""
    # Warm-up (para evitar medir inicialización/compilación)
    for _ in range(n_warmup):
        fn()
    cp.cuda.Stream.null.synchronize()

    times = []
    for _ in range(n_repeat):
        start = cp.cuda.Event()
        end = cp.cuda.Event()
        start.record()
        fn()
        end.record()
        end.synchronize()
        times.append(cp.cuda.get_elapsed_time(start, end))  # ms
    return np.array(times, dtype=np.float64)

def show_stats(label, times_ms):
    print(
        f"{label}: mean={times_ms.mean():.3f} ms | std={times_ms.std():.3f} ms | "
        f"min={times_ms.min():.3f} ms | max={times_ms.max():.3f} ms" )

### CPU: plain and numpy

In [25]:
import numpy as np
from numba import njit

@njit
def grade2_vector(x, y, a, b, c):
    z = np.empty(x.size, dtype=np.float32)
    for i in range(x.size):
        z[i] = a * x[i] * x[i] + b * y[i] + c
    return z

def grade2_ufunc(x, y, a, b, c):
    return a * x**2 + b * y + c

# Warm-up (muy importante para Numba)
_ = grade2_vector(a_cpu[:10], b_cpu[:10], a, b, c)

# Guardar salida (si el profesor lo llama c_cpu, lo dejamos así)
c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)

print("CPU numba njit:")
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

print("CPU numpy ufunc:")
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

print("CPU numpy expresión directa:")
%timeit -n 5 -r 2 (a * a_cpu**2 + b * b_cpu + c)

CPU numba njit:
2.56 ms ± 99.5 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
CPU numpy ufunc:
8.98 ms ± 331 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
CPU numpy expresión directa:
8.62 ms ± 55.2 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


Celda 2 — CuPy con copia CPU↔GPU (baseline GPU)

In [16]:
# Ufunc en CuPy tipo "elementwise"
grade2_cupy_ufunc = cp.ElementwiseKernel(
    in_params='float32 x, float32 y, float32 a, float32 b, float32 c',
    out_params='float32 z',
    operation='z = a * x * x + b * y + c;',
    name='grade2_cupy_ufunc'
)

def cupy_with_copy():
    # Copia CPU->GPU
    xg = cp.asarray(a_cpu)
    yg = cp.asarray(b_cpu)
    # Compute en GPU
    zg = grade2_cupy_ufunc(xg, yg, a, b, c)
    # Copia GPU->CPU (incluida en el tiempo “con copia”)
    _ = cp.asnumpy(zg)

# Warm-up
cupy_with_copy()
cp.cuda.Stream.null.synchronize()


# Medición
times = gpu_time_ms(cupy_with_copy, n_repeat=10, n_warmup=3)
show_stats("CuPy ufunc CON copia (CPU<->GPU incluido)", times)
times = gpu_time_ms(cupy_with_copy, n_repeat=10, n_warmup=3)

CuPy ufunc CON copia (CPU<->GPU incluido): mean=7.128 ms | std=0.052 ms | min=7.074 ms | max=7.246 ms


 Celda 3 — CuPy sin copia (crear directamente en GPU)

In [30]:
def cupy_no_copy():
    # Datos en GPU directamente (sin transferencia desde CPU)
    xg = cp.random.random(size).astype(cp.float32)
    yg = cp.random.random(size).astype(cp.float32)
    zg = grade2_cupy_ufunc(xg, yg, a, b, c)
    # NO hacemos asnumpy aquí para no contar copias
    return zg

# Warm-up
_ = cupy_no_copy()
cp.cuda.Stream.null.synchronize()

#Medición
times = gpu_time_ms(cupy_no_copy, n_repeat=10, n_warmup=3)
show_stats("CuPy SIN copia", times)

CuPy SIN copia: mean=1.666 ms | std=0.002 ms | min=1.661 ms | max=1.668 ms


Celda 4 — Numba en GPU usando @vectorize(target='cuda')

In [31]:

@vectorize(['float32(float32, float32, float32, float32, float32)'], target='cuda')
def grade2_numba_cuda(x, y, a, b, c):
    return a * x * x + b * y + c

def numba_with_copy():
    # Si le pasas NumPy arrays, N    # Si le pasas NumPy arrays, Numba hace copias automáticas host<->device
    _ = grade2_numba_cuda(a_cpu, b_cpu, a, b, c)

# Warm-up
numba_with_copy()
cp.cuda.Stream.null.synchronize()

#Medición
times = gpu_time_ms(numba_with_copy, n_repeat=10, n_warmup=3)
show_stats("Numba CON copia", times)

Numba CON copia: mean=8.851 ms | std=1.213 ms | min=7.555 ms | max=10.744 ms


Celda 5 — Numba @vectorize(target='cuda') SIN CONTAR COPIA

In [32]:
# Copia manual FUERA del timing
xg = cp.asarray(a_cpu)
yg = cp.asarray(b_cpu)
cp.cuda.Stream.null.synchronize()

def numba_no_copy_counted():
    # al pasar arrays que viven en GPU (cupy), Numba puede operar via __cuda_array_interface__
    _ = grade2_numba_cuda(xg, yg, a, b, c)

# Warm-up
numba_no_copy_counted()
cp.cuda.Stream.null.synchronize()

#Medición
times = gpu_time_ms(numba_no_copy_counted, n_repeat=10, n_warmup=3)
show_stats("Numba SIN copia", times)

Numba SIN copia: mean=1.612 ms | std=0.576 ms | min=1.129 ms | max=2.460 ms


CELDA 8 — Verificación de resultados (CPU vs GPU)

In [24]:
# Referencia CPU (NumPy ufunc)
z_ref = grade2_ufunc(a_cpu, b_cpu, a, b, c).astype(np.float32)

# CuPy con copia: obtenemos salida CPU y comparamos
xg2 = cp.asarray(a_cpu)
yg2 = cp.asarray(b_cpu)
zg2 = grade2_cupy_ufunc(xg2, yg2, a, b, c)
z_cupy = cp.asnumpy(zg2).astype(np.float32)

print("CuPy OK?:", np.allclose(z_ref, z_cupy, rtol=1e-5, atol=1e-6))

# Numba CUDA (con# Numba CUDA (con arrays CPU, devuelve resultado en host normalmente)
z_numba = grade2_numba_cuda(a_cpu, b_cpu, a, b, c).astype(np.float32)

CuPy OK?: True


Celda 6 — Conclusiones (texto)


## Conclusiones (CPU vs GPU: CuPy y Numba con/sin copia)

### 1) Resultados en CPU
- **Numba `@njit` (bucle)**: **2.56 ms**  
- **NumPy ufunc**: **8.98 ms**  
- **Expresión NumPy directa** (`a*x**2 + b*y + c`): **8.62 ms**

En este caso, **Numba en CPU es ~3.4× más rápido** que NumPy:
- 8.62 / 2.56 ≈ **3.37×**
Esto es coherente porque `@njit` compila el bucle a código máquina y elimina el overhead de Python, mientras que NumPy, aunque vectoriza, puede incurrir en temporales/overhead y no siempre supera a un bucle compilado eficiente.

### 2) Resultados en GPU con CuPy (ufunc element-wise)
- **CuPy CON copia (CPU↔GPU incluido)**: **7.128 ms**
- **CuPy SIN copia (todo en GPU)**: **1.666 ms**

Se observa claramente el efecto de la transferencia:
- La versión **SIN copia es ~4.3× más rápida** que la versión con copia:  
  7.128 / 1.666 ≈ **4.28×**
Esto confirma que, para operaciones element-wise simples, el coste de **copiar datos** entre CPU y GPU puede dominar el tiempo total. Cuando los datos se crean y se mantienen en GPU, el cálculo paralelizado en la GPU se aprovecha mucho mejor.

### 3) Resultados en GPU con Numba (`@vectorize(target='cuda')`)
- **Numba CON copia (automática)**: **8.851 ms**
- **Numba SIN contar copia (datos ya en GPU)**: **1.612 ms**

De nuevo, la diferencia entre “con copia” y “sin copia” es grande:
- 8.851 / 1.612 ≈ **5.49×**
Esto encaja con lo esperado: cuando pasamos arrays de CPU, Numba debe gestionar transferencias (host↔device). Si los datos ya están en GPU y se mide solo el cómputo, se aprecia el rendimiento real del kernel/ufunc en GPU.

### 4) Comparación global e interpretación
- **Mejor tiempo absoluto observado**: GPU sin copia (**~1.6–1.7 ms**)  
- **CPU más rápida**: Numba `@njit` (**2.56 ms**)  
- **GPU con copia** queda más cerca de CPU: CuPy con copia (**7.13 ms**) y Numba con copia (**8.85 ms**)

Conclusión principal:
> **La GPU acelera el cálculo, pero solo se obtiene una mejora clara si se minimizan las transferencias CPU↔GPU.**  
Para tareas simples por elemento (como `a*x^2 + b*y + c`), el “cuello de botella” suele ser la transferencia de datos. Por eso, el mayor beneficio aparece cuando los datos se generan o permanecen en GPU (sin copia).

### 5) Nota sobre variabilidad (desviación estándar)
En Numba SIN copia aparece una desviación mayor (**std ≈ 0.576 ms**) que en CuPy SIN copia (**std ≈ 0.002 ms**). Esto puede deberse a:
- efectos de inicialización/gestión interna de memoria,
- variabilidad del sistema,
- y/o a que el pipeline de- y/o a que el pipeline de ejecución asíncrona y sincronización afecte a algunas repeticiones.